<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Autoencoders_and_Variational_Autoencoders_(VAEs).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, latent_dim * 2)  # Mean and log-variance
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        mean_logvar = self.encoder(x)
        mean, logvar = mean_logvar[:, :latent_dim], mean_logvar[:, latent_dim:]
        std = torch.exp(0.5 * logvar)
        z = mean + std * torch.randn_like(std)
        x_reconstructed = self.decoder(z)
        return x_reconstructed, mean, logvar

def vae_loss(x_reconstructed, x, mean, logvar):
    reconstruction_loss = nn.BCELoss(reduction='sum')(x_reconstructed, x)
    kl_divergence = -0.5 * torch.sum(1 + logvar - mean.pow(2) - logvar.exp())
    return reconstruction_loss + kl_divergence

input_dim = 28 * 28  # MNIST images flattened
latent_dim = 10

model = VAE(input_dim, latent_dim)  # Removed .to('cuda')
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Dummy data for training
x = torch.rand(32, input_dim)  # Removed .to('cuda')

for epoch in range(50):
    optimizer.zero_grad()
    x_reconstructed, mean, logvar = model(x)
    loss = vae_loss(x_reconstructed, x, mean, logvar)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")